In [1]:
from matchmaker import matchmaker

In [ ]:
engine = matchmaker.MatchingEngine()
engine.load_interactions("data/swipes.csv", 
  decider_col='decidermemberid', 
  other_col='othermemberid', 
  like_col='like', 
  timestamp_col='timestamp')

In [3]:
engine.interactions_df.head()

,decider_id,other_id,interaction_type,timestamp
0,3847776,3227524,1,2021-01-01 00:00:06
1,608590,519321,0,2021-01-01 00:00:06
2,397116,453914,0,2021-01-01 00:00:06
3,3847776,1269455,1,2021-01-01 00:00:06
4,1630969,347909,0,2021-01-01 00:00:23
